In [62]:
import numpy as np
import pandas as pd

from sklearn.utils import shuffle

In [74]:
data = pd.read_csv("hasznaltautohu_db.csv", encoding="ISO-8859-1")
brand_cat = None
vehicle_type_cat = None 
fuel_type_cat = None


data["BRAND"] = data["BRAND"].astype("category")
brand_cat = data["BRAND"].cat.categories.to_list()
data["BRAND"] = data["BRAND"].cat.codes

data["JARMU JELLEGE"] = data["JARMU JELLEGE"].astype("category")
vehicle_type_cat = data["JARMU JELLEGE"].cat.categories.to_list()
data["JARMU JELLEGE"] = data["JARMU JELLEGE"].cat.codes

data["UZEMANYAGTIPUS"] = data["UZEMANYAGTIPUS"].astype("category")
fuel_type_cat = data["UZEMANYAGTIPUS"].cat.categories.to_list()
data["UZEMANYAGTIPUS"] = data["UZEMANYAGTIPUS"].cat.codes

data.head()

,BRAND,JARMU JELLEGE,UZEMANYAGTIPUS,EVJARAT,HENGERURTARTALOM (CM3),TELJESITMENY (LE),FUTOTT KILOMETER,AR (HUF)
0,68,0,2,1996,6871,260,803039,1300000
1,75,0,2,1997,5958,132,217236,1500000
2,97,0,2,2002,10640,344,766500,1500000
3,83,0,2,2004,3989,136,656660,1650000
4,50,0,2,2011,3920,177,351968,2100000


In [64]:
data = shuffle(data, random_state=42)
data.reset_index(inplace=True, drop=True)
data.head()

,BRAND,JARMU JELLEGE,UZEMANYAGTIPUS,EVJARAT,HENGERURTARTALOM (CM3),TELJESITMENY (LE),FUTOTT KILOMETER,AR (HUF)
0,110,2,3,2020,0,513,9700,20990000
1,116,2,2,1999,1896,116,280000,195000
2,116,2,2,2012,1968,140,267097,4999000
3,38,1,2,2005,1998,101,399000,960000
4,91,1,2,2007,2464,120,330000,2500000


In [65]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [66]:
model = None

dfX, dfY = data.loc[:, :"FUTOTT KILOMETER"], data["AR (HUF)"]
X, y = dfX.to_numpy(), dfY.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
def evaluate_model():
    y_pred = model.predict(X_test)
    print("Error: {}".format(mean_squared_error(y_test, y_pred)))

In [68]:
model = LinearRegression()
model.fit(X_train, y_train)

evaluate_model()

Error: 12665495067957.303


In [69]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

evaluate_model()

Error: 4452042099541.908


In [70]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

evaluate_model()

Error: 2659459382200.084


# Teszt más oldalakon talált autókkal

In [79]:
def make_car(brand, vhc_type, fuel_type, year, cc, hp, km):
    brand_i = brand_cat.index(brand)
    vehicle_type_i = vehicle_type_cat.index(vhc_type)
    fuel_type_i = fuel_type_cat.index(fuel_type)

    return [[brand_i, vehicle_type_i, fuel_type_i, year, cc, hp, km],]

In [80]:
# Suzuki Swift 1998 (elvárt: ~350-450k) 

car = make_car("SUZUKI", "szemelyauto", "Benzin", 1998, 1298, 67, 94000)
car_price = model.predict(car)

print(car_price)

[357860.]


In [81]:
# TESLA model3 (elvárt 16M-22M)

car = make_car("TESLA", "szemelyauto", "Elektromos", 2021, 0, 260, 300)
car_price = model.predict(car)

print(car_price)

[17467247.02020202]


In [84]:
# IVECO 35 2011 (elvárt 2.2-3.2M)

car = make_car("IVECO", "kishaszongepjarmu", "Dízel", 2011, 2287, 126, 113000)
car_price = model.predict(car)

print(car_price)

[4063689.92]


In [85]:
# BMW i8 (elvárt 22M - 29M)

car = make_car("BMW", "szemelyauto", "Hibrid", 2016, 1499, 231, 77352)
car_price = model.predict(car)

print(car_price)

[22306730.]


In [86]:
# Scania 2008 körüli nyergesvontató 700e km-el (elvárt ~6M)

car = make_car("SCANIA", "haszongepjarmu", "Dízel", 2008, 12000, 309, 700000)
car_price = model.predict(car)

print(car_price)

[9985380.]
